# Bond Valuation Analysis

## Methodology

Bond valuation was performed using the standard present value approach, discounting all future cash flows at the risk-free rate:

$Bond\;Value = \sum_{t=1}^{n}\frac{C}{(1+r)^t} + \frac{F}{(1+r)^n}$

Where:
- $C$ = periodic coupon payment
- $F$ = face value
- $r$ = discount rate per period
- $n$ = number of periods

For interest rate sensitivity analysis, we calculated:

- **Macaulay Duration**: The weighted average time to receive cash flows
- **Modified Duration**: Approximates the percentage price change for a 1% change in yield

# Part I: Walmart Inc. Bond Analysis

## Overview

This analysis examines a Walmart Inc. bond maturing on September 22, 2031, with a 1.80% coupon rate. Using a risk-free rate of 4.27%, we've calculated its fair value, yield to maturity, duration, and interest rate sensitivity to understand its investment characteristics.


In [15]:
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

In [18]:
face_value = 1000  # Face value of the bond
coupon_rate = 0.018  # 1.80% annual coupon rate
risk_free_rate = 0.0427  # Given risk-free rate
current_date = date(2025, 4, 19)
maturity_date = date(2031, 9, 22)
call_date = date(2031, 6, 22)
current_price = 85.99 * 10  # Converting percentage to actual price

years_to_maturity = (maturity_date - current_date).days / 365.25
periods_to_maturity = int(np.ceil(years_to_maturity * 2))  # Semiannual periods

# Determine exact period fractions (for more accuracy)
# This handles partial periods if the next payment date isn't exactly in 6 months
# For simplicity, we'll use whole periods for this analysis

def calculate_bond_price(face_value, coupon_rate, periods, discount_rate):
    """
    Calculate the present value of a bond using the textbook formula
    for semiannual coupon payments
    """

    semiannual_coupon = face_value * coupon_rate / 2
    semiannual_rate = discount_rate / 2

    present_value = 0
    
    # Sum the PV of all coupon payments
    for t in range(1, periods + 1):
        present_value += semiannual_coupon / ((1 + semiannual_rate) ** t)

    present_value += face_value / ((1 + semiannual_rate) ** periods)
    
    return present_value

bond_value = calculate_bond_price(face_value, coupon_rate, periods_to_maturity, risk_free_rate)


def calculate_ytm(price, face_value, coupon_rate, periods):
    """
    Calculate yield to maturity using a simple trial and error approach
    as mentioned in the textbook
    """
    # Initial guess
    lower_bound = 0.0001
    upper_bound = 0.2  # 20% as a high upper bound
    tolerance = 0.0000001
    
    while (upper_bound - lower_bound) > tolerance:
        mid_rate = (lower_bound + upper_bound) / 2
        price_at_mid = calculate_bond_price(face_value, coupon_rate, periods, mid_rate)
        
        if price_at_mid > price:
            lower_bound = mid_rate
        else:
            upper_bound = mid_rate
    
    return (lower_bound + upper_bound) / 2

ytm = calculate_ytm(current_price, face_value, coupon_rate, periods_to_maturity)

def calculate_duration(face_value, coupon_rate, periods, discount_rate):
    """
    Calculate Macaulay duration using the formula in the textbook
    """
    semiannual_coupon = face_value * coupon_rate / 2
    semiannual_rate = discount_rate / 2
    
    pv = calculate_bond_price(face_value, coupon_rate, periods, discount_rate)
    
    weighted_pv_sum = 0
    
    # Sum the weighted PV of all coupon payments
    for t in range(1, periods + 1):
        time_weight = t / 2

        payment_pv = semiannual_coupon / ((1 + semiannual_rate) ** t)
        weighted_pv_sum += time_weight * payment_pv
    
    weighted_pv_sum += (periods / 2) * (face_value / ((1 + semiannual_rate) ** periods))

    macaulay_duration = weighted_pv_sum / pv
    modified_duration = macaulay_duration / (1 + semiannual_rate)
    
    return macaulay_duration, modified_duration

macaulay_duration, modified_duration = calculate_duration(
    face_value, coupon_rate, periods_to_maturity, risk_free_rate)

print(f"WMT 1.8% 09/22/2031 Analysis (As of {current_date})")
print("-" * 50)
print(f"Face Value: ${face_value:.2f}")
print(f"Coupon Rate: {coupon_rate*100:.2f}%")
print(f"Risk-Free Rate: {risk_free_rate*100:.2f}%")
print(f"Years to Maturity: {years_to_maturity:.2f} years")
print(f"Number of Semiannual Periods: {periods_to_maturity}")
print("-" * 50)
print(f"Fair Value (using risk-free rate): ${bond_value:.2f}")
print(f"Current Market Price: ${current_price:.2f}")
print(f"Yield to Maturity: {ytm*100:.4f}%")
print("-" * 50)
print(f"Macaulay Duration: {macaulay_duration:.2f} years")
print(f"Modified Duration: {modified_duration:.4f}")
print(f"Price change for 1% increase in yield: ${-modified_duration * bond_value * 0.01:.2f}")
print("-" * 50)

if current_price < bond_value:
    discount = bond_value - current_price
    discount_percent = (discount / bond_value) * 100
    print(f"Bond is trading at a discount of €{discount:.2f} ({discount_percent:.2f}%) to fair value")
elif current_price > bond_value:
    premium = current_price - bond_value
    premium_percent = (premium / bond_value) * 100
    print(f"Bond is trading at a premium of €{premium:.2f} ({premium_percent:.2f}%) to fair value")
else:
    print(f"Bond is trading at fair value")

WMT 1.8% 09/22/2031 Analysis (As of 2025-04-19)
--------------------------------------------------
Face Value: $1000.00
Coupon Rate: 1.80%
Risk-Free Rate: 4.27%
Years to Maturity: 6.43 years
Number of Semiannual Periods: 13
--------------------------------------------------
Fair Value (using risk-free rate): $861.09
Current Market Price: $859.90
Yield to Maturity: 4.2930%
--------------------------------------------------
Macaulay Duration: 6.13 years
Modified Duration: 6.0018
Price change for 1% increase in yield: $-51.68
--------------------------------------------------
Bond is trading at a discount of €1.19 (0.14%) to fair value


## Key Metrics and Results

**Walmart Inc. 1.80% Bond (September 22, 2031)**

| Parameter | Value |
|-----------|-------|
| Face Value | $1,000.00 |
| Coupon Rate | 1.80% |
| Coupon Frequency | Semi-annual |
| Years to Maturity | 6.43 |
| Risk-Free Rate | 4.27% |
| Calculated Fair Value | $861.09 |
| Current Market Price | $859.90 |
| Yield to Maturity | 4.29% |
| Macaulay Duration | 6.13 years |
| Modified Duration | 6.00 |

## Analysis of Results

The Walmart bond trades at a modest discount of 0.14% to its calculated fair value ($859.90 vs. $861.09), suggesting the market is pricing this bond with a risk premium very slightly above our risk-free rate assumption of 4.27%. The bond's significant discount to par value (approximately 14%) primarily reflects its below-market coupon rate of 1.80% in the current interest rate environment.

The yield to maturity of 4.29% indicates the total return an investor would receive if they purchased the bond at the current market price and held it until maturity. This yield exceeds the coupon rate by 2.49 percentage points, which explains the substantial discount to par value. In essence, the market is compensating for the opportunity cost of holding this low-coupon bond when higher rates are available elsewhere.

With a modified duration of 6.00, the bond exhibits moderate interest rate sensitivity – a 1% increase in yields would reduce the bond's price by approximately 6% or $51.68. This duration is somewhat shorter than the time to maturity (6.43 years) due to the effect of receiving coupon payments before maturity.

## Investment Implications

The Walmart bond offers a stable investment option from a company with strong fundamentals and a resilient business model. As one of the world's largest retailers with significant cash flows and a strong balance sheet, Walmart presents relatively low credit risk despite operating in the competitive retail sector.

The bond's current pricing reflects market expectations regarding interest rates and Walmart's credit quality. The small difference between the calculated fair value and market price suggests efficient pricing in the corporate bond market.

For investors with a medium-term investment horizon of 6-7 years, this bond provides a reasonable yield with moderate interest rate risk. The semi-annual coupon payments, while modest, offer periodic income that can be reinvested.

In the context of the current interest rate environment, investors should consider:

1. **Interest Rate Outlook**: If rates are expected to rise further, the bond's price could face additional downward pressure due to its 6.00 modified duration.

2. **Reinvestment Risk**: The low 1.80% coupon creates relatively lower reinvestment risk compared to higher-coupon bonds, as there's less dependence on reinvesting coupon payments at attractive rates.

3. **Tax Considerations**: For taxable accounts, the bond's discount to par offers a tax advantage as the return comes partially from capital appreciation (taxed as capital gains) rather than solely from coupon income (taxed as ordinary income).

4. **Liquidity**: The bond's high liquidity score suggests it can be readily sold if needed, providing flexibility for portfolio adjustments.

The Walmart bond represents a relatively conservative fixed-income investment with a yield that adequately compensates for its duration and credit risk in the current market context.

# Part II: Stellantis N.V. Bond Analysis

## Overview

This analysis examines a Stellantis N.V. medium-term note maturing in 2034 with a 4.00% coupon rate. Using a risk-free rate of 4.27%, we've calculated its fair value, yield to maturity, duration, and interest rate sensitivity to understand its investment characteristics.

In [19]:
face_value = 1000  # Face value of the bond in EUR
coupon_rate = 0.04  # 4.00% annual coupon rate
risk_free_rate = 0.0427  # Given risk-free rate
current_date = date(2025, 4, 19)
maturity_date = date(2034, 3, 19)
current_price = 989.50  # 98.95% of face value

# Calculate time periods
years_to_maturity = (maturity_date - current_date).days / 365.25

# Calculate the exact time to each cash flow
coupon_dates = []
for year in range(2026, 2035):
    coupon_dates.append(date(year, 3, 19))

years_to_cashflows = [(payment_date - current_date).days / 365.25 
                       for payment_date in coupon_dates]

# Annual coupon payment
annual_coupon = face_value * coupon_rate

# Calculate bond present value
def calculate_bond_price(face_value, coupon_rate, discount_rate, years_to_cashflows):
    """
    Calculate the present value of a bond with annual coupon payments
    """
    annual_coupon = face_value * coupon_rate
    present_value = 0
    
    # Sum the PV of all coupon payments
    for i, years in enumerate(years_to_cashflows[:-1]):
        present_value += annual_coupon / ((1 + discount_rate) ** years)
    
    present_value += (annual_coupon + face_value) / ((1 + discount_rate) ** years_to_cashflows[-1])
    
    return present_value

bond_value = calculate_bond_price(face_value, coupon_rate, risk_free_rate, years_to_cashflows)


def calculate_ytm(price, face_value, coupon_rate, years_to_cashflows):
    """
    Calculate yield to maturity using binary search method
    """
    # Initial guess boundaries
    lower_bound = 0.0001
    upper_bound = 0.2  # 20% as a high upper bound
    tolerance = 0.0000001
    
    while (upper_bound - lower_bound) > tolerance:
        mid_rate = (lower_bound + upper_bound) / 2
        price_at_mid = calculate_bond_price(face_value, coupon_rate, mid_rate, years_to_cashflows)
        
        if price_at_mid > price:
            lower_bound = mid_rate
        else:
            upper_bound = mid_rate
    
    return (lower_bound + upper_bound) / 2

ytm = calculate_ytm(current_price, face_value, coupon_rate, years_to_cashflows)

def calculate_duration(face_value, coupon_rate, discount_rate, years_to_cashflows):
    """
    Calculate Macaulay duration for a bond with annual payments
    """
    annual_coupon = face_value * coupon_rate
    
    pv = calculate_bond_price(face_value, coupon_rate, discount_rate, years_to_cashflows)
    
    weighted_pv_sum = 0
    
    # Sum the weighted PV of all coupon payments
    for i, years in enumerate(years_to_cashflows[:-1]):
        payment_pv = annual_coupon / ((1 + discount_rate) ** years)
        
        weighted_pv_sum += years * payment_pv
    
    # sum of the weighted PV of the final coupon plus face value
    weighted_pv_sum += years_to_cashflows[-1] * (annual_coupon + face_value) / ((1 + discount_rate) ** years_to_cashflows[-1])

    macaulay_duration = weighted_pv_sum / pv
    modified_duration = macaulay_duration / (1 + discount_rate)
    
    return macaulay_duration, modified_duration

macaulay_duration, modified_duration = calculate_duration(
    face_value, coupon_rate, risk_free_rate, years_to_cashflows)

print(f"STELLANTIS 24/34 MTN Analysis (As of {current_date})")
print("-" * 60)
print(f"Face Value: €{face_value:.2f}")
print(f"Coupon Rate: {coupon_rate*100:.2f}%")
print(f"Risk-Free Rate: {risk_free_rate*100:.2f}%")
print(f"Years to Maturity: {years_to_maturity:.2f} years")
print(f"Payment Frequency: Annual")
print("-" * 60)
print(f"Fair Value (using risk-free rate): €{bond_value:.2f}")
print(f"Current Market Price: €{current_price:.2f}")
print(f"Yield to Maturity: {ytm*100:.4f}%")
print("-" * 60)
print(f"Macaulay Duration: {macaulay_duration:.2f} years")
print(f"Modified Duration: {modified_duration:.4f}")
print(f"Price change for 1% increase in yield: €{-modified_duration * bond_value * 0.01:.2f}")
print("-" * 60)

# Market value comparison
if current_price < bond_value:
    discount = bond_value - current_price
    discount_percent = (discount / bond_value) * 100
    print(f"Bond is trading at a discount of €{discount:.2f} ({discount_percent:.2f}%) to fair value")
elif current_price > bond_value:
    premium = current_price - bond_value
    premium_percent = (premium / bond_value) * 100
    print(f"Bond is trading at a premium of €{premium:.2f} ({premium_percent:.2f}%) to fair value")
else:
    print(f"Bond is trading at fair value")

STELLANTIS 24/34 MTN Analysis (As of 2025-04-19)
------------------------------------------------------------
Face Value: €1000.00
Coupon Rate: 4.00%
Risk-Free Rate: 4.27%
Years to Maturity: 8.91 years
Payment Frequency: Annual
------------------------------------------------------------
Fair Value (using risk-free rate): €983.68
Current Market Price: €989.50
Yield to Maturity: 4.1894%
------------------------------------------------------------
Macaulay Duration: 7.63 years
Modified Duration: 7.3197
Price change for 1% increase in yield: €-72.00
------------------------------------------------------------
Bond is trading at a premium of €5.82 (0.59%) to fair value


## Key Metrics and Results

**Stellantis N.V. 4.00% Medium-Term Note (2034)**

| Parameter | Value |
|-----------|-------|
| Face Value | €1,000.00 |
| Coupon Rate | 4.00% |
| Coupon Frequency | Annual |
| Years to Maturity | 8.91 |
| Risk-Free Rate | 4.27% |
| Calculated Fair Value | €983.68 |
| Current Market Price | €989.50 |
| Yield to Maturity | 4.19% |
| Macaulay Duration | 7.63 years |
| Modified Duration | 7.32 |

## Analysis of Results

The Stellantis bond trades at a premium of 0.59% to its calculated fair value (€989.50 vs. €983.68), indicating the market perceives slightly less risk than what would be implied by our risk-free rate assumption of 4.27%. The bond's closer proximity to par value reflects its coupon rate (4.00%) being much closer to current market yields.

The yield to maturity of 4.19% indicates the total return an investor would receive if they purchased the bond at the current market price and held it until maturity. This yield is only slightly higher than the coupon rate, explaining why the bond's price is relatively close to its par value.

With a Macaulay duration of 7.63 years and a modified duration of 7.32, the bond exhibits significant interest rate sensitivity. A 1% increase in yields would decrease the bond's price by approximately 7.3% or €72.00. The longer duration reflects both the bond's longer maturity (8.91 years) and its annual (rather than semi-annual) coupon payment structure.

## Investment Implications

The Stellantis bond offers exposure to one of the world's largest automotive manufacturers, a company formed from the merger of Fiat Chrysler Automobiles and the PSA Group. The automotive industry faces significant transition challenges with the shift toward electric vehicles and changing mobility patterns, which introduces some sector-specific risks.

The bond's pricing indicates market confidence in Stellantis's ability to navigate these industry changes and meet its debt obligations. The premium to calculated fair value suggests investors may be placing a slightly lower risk premium on this bond than our risk-free rate would imply.

For investors with a longer-term investment horizon of 8-9 years, this bond provides reasonable yield with the following considerations:

1. **Interest Rate Sensitivity**: With a modified duration of 7.32, this bond carries substantial interest rate risk. In a rising rate environment, investors could face meaningful price declines.

2. **Currency Exposure**: As a EUR-denominated bond, it introduces currency risk for non-eurozone investors. Currency fluctuations could significantly impact total returns when converted back to the investor's base currency.

3. **Industry Transition Risk**: The automotive sector is undergoing profound transformation with electrification, autonomous driving, and changing ownership models. Stellantis's success in navigating these changes will impact its credit quality over the bond's lifetime.

4. **Reinvestment Risk**: The 4.00% annual coupon creates moderate reinvestment risk, as these payments will need to be reinvested at prevailing rates over the next 8.91 years.

5. **European Exposure**: The bond provides exposure to European economic conditions, as Stellantis has significant operations across Europe.

The bond's pricing, with a yield to maturity of 4.19% compared to the 4.27% risk-free rate, suggests the market views Stellantis's credit quality favorably relative to its industry position and the general macroeconomic environment.

For investors seeking exposure to the European corporate bond market with a medium-to-long-term investment horizon, this bond represents a reasonable option that balances yield with the credit fundamentals of a major global automotive manufacturer.